# Notebook for developing models

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

## Data preparation for model training

### Importing data

In [2]:
# read in data
train = pd.read_csv("train.csv", delimiter=",")
test = pd.read_csv("test.csv", delimiter=",")
X_additional = pd.read_csv("additionalAttributes.csv", delimiter=",").drop("Unnamed: 0", axis=1)
X = train.drop(["formation_energy_ev_natom", "bandgap_energy_ev", "id"], axis=1)
y_fe = train.formation_energy_ev_natom
y_be = train.bandgap_energy_ev
X_full = pd.concat([X, X_additional], axis=1)

In [3]:
X_full

,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,...,distGaO,distInAl,distInGa,distInO,elInt,qAl,qGa,qIn,qO,xEq
0,33,80.0,0.6250,0.3750,0.0000,9.9523,8.5513,9.1775,90.0026,90.0023,...,2.025,NaN,NaN,NaN,-41.086176,0.349004,0.345264,NaN,-0.231734,4.896453
1,194,80.0,0.6250,0.3750,0.0000,6.1840,6.1838,23.6287,90.0186,89.9980,...,1.775,0.025,0.025,1.775,-38.554098,0.344690,0.347084,NaN,-0.230392,4.905702
2,227,40.0,0.8125,0.1875,0.0000,9.7510,5.6595,13.9630,90.9688,91.1228,...,1.825,0.025,0.025,1.825,-33.957596,0.343019,0.339820,NaN,-0.228280,4.925995
3,167,30.0,0.7500,0.0000,0.2500,5.0036,5.0034,13.5318,89.9888,90.0119,...,0.025,2.925,0.025,0.025,-35.763321,0.337616,NaN,0.375759,-0.231435,4.898196
4,194,80.0,0.0000,0.6250,0.3750,6.6614,6.6612,24.5813,89.9960,90.0006,...,2.025,0.025,3.625,2.025,-36.645650,NaN,0.342521,0.359175,-0.232511,4.869110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,33,40.0,0.7500,0.2500,0.0000,4.9469,8.5014,9.1298,90.0038,90.0023,...,1.775,0.025,0.025,1.775,-39.262887,0.346869,0.345489,NaN,-0.231016,4.905482
2396,167,30.0,0.4167,0.5833,0.0000,4.9566,4.9562,13.4178,89.9938,90.0075,...,2.025,0.025,0.025,2.025,-35.323868,0.345140,0.342118,NaN,-0.228918,4.928134
2397,206,80.0,0.4375,0.5625,0.0000,9.2204,9.2200,9.2199,90.0047,90.0046,...,1.975,0.025,0.025,1.975,-38.566520,0.364599,0.330046,NaN,-0.230109,4.873503
2398,33,80.0,0.3125,0.1875,0.5000,10.6529,9.0954,9.7210,90.0015,89.9996,...,2.125,3.375,3.525,2.125,-39.874295,0.335601,0.329533,0.372616,-0.235314,4.842888


### Remove nan values From data

In [4]:
X_full.columns

Index(['spacegroup', 'number_of_total_atoms', 'percent_atom_al',
       'percent_atom_ga', 'percent_atom_in', 'lattice_vector_1_ang',
       'lattice_vector_2_ang', 'lattice_vector_3_ang',
       'lattice_angle_alpha_degree', 'lattice_angle_beta_degree',
       'lattice_angle_gamma_degree', 'Vatom', 'cAlGa', 'cAlIn', 'cAlO',
       'cGaAl', 'cGaIn', 'cGaO', 'cInAl', 'cInGa', 'cInO', 'distAlGa',
       'distAlIn', 'distAlO', 'distGaAl', 'distGaIn', 'distGaO', 'distInAl',
       'distInGa', 'distInO', 'elInt', 'qAl', 'qGa', 'qIn', 'qO', 'xEq'],
      dtype='object')

* If distance in Nan, it is reasonable to set them to very large value as the atoms are infinitely far from each other
* If Coordination number is Nan then it is reasonable to set it as 0, as no atoms are in vicinity of chosen atom
* If any of the charges have Nan values it is set to 0

In [5]:
fillValues = {'cAlGa':0, 
              'cAlIn':0, 
              'cAlO':0,
              'cGaAl':0, 
              'cGaIn':0, 
              'cGaO':0, 
              'cInAl':0, 
              'cInGa':0, 
              'cInO':0,
              'distAlGa':9999999,
              'distAlIn':9999999,
              'distAlO':9999999,
              'distGaAl':9999999,
              'distGaIn':9999999,
              'distGaO':9999999,
              'distInAl':9999999,
              'distInGa':9999999,
              'distInO':9999999,
              'qAl':0, 
              'qGa':0, 
              'qIn':0, 
              'qO':0}

In [6]:
X_full = X_full.fillna(value=fillValues)
X_full

,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,...,distGaO,distInAl,distInGa,distInO,elInt,qAl,qGa,qIn,qO,xEq
0,33,80.0,0.6250,0.3750,0.0000,9.9523,8.5513,9.1775,90.0026,90.0023,...,2.025,9999999.000,9999999.000,9999999.000,-41.086176,0.349004,0.345264,0.000000,-0.231734,4.896453
1,194,80.0,0.6250,0.3750,0.0000,6.1840,6.1838,23.6287,90.0186,89.9980,...,1.775,0.025,0.025,1.775,-38.554098,0.344690,0.347084,0.000000,-0.230392,4.905702
2,227,40.0,0.8125,0.1875,0.0000,9.7510,5.6595,13.9630,90.9688,91.1228,...,1.825,0.025,0.025,1.825,-33.957596,0.343019,0.339820,0.000000,-0.228280,4.925995
3,167,30.0,0.7500,0.0000,0.2500,5.0036,5.0034,13.5318,89.9888,90.0119,...,0.025,2.925,0.025,0.025,-35.763321,0.337616,0.000000,0.375759,-0.231435,4.898196
4,194,80.0,0.0000,0.6250,0.3750,6.6614,6.6612,24.5813,89.9960,90.0006,...,2.025,0.025,3.625,2.025,-36.645650,0.000000,0.342521,0.359175,-0.232511,4.869110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,33,40.0,0.7500,0.2500,0.0000,4.9469,8.5014,9.1298,90.0038,90.0023,...,1.775,0.025,0.025,1.775,-39.262887,0.346869,0.345489,0.000000,-0.231016,4.905482
2396,167,30.0,0.4167,0.5833,0.0000,4.9566,4.9562,13.4178,89.9938,90.0075,...,2.025,0.025,0.025,2.025,-35.323868,0.345140,0.342118,0.000000,-0.228918,4.928134
2397,206,80.0,0.4375,0.5625,0.0000,9.2204,9.2200,9.2199,90.0047,90.0046,...,1.975,0.025,0.025,1.975,-38.566520,0.364599,0.330046,0.000000,-0.230109,4.873503
2398,33,80.0,0.3125,0.1875,0.5000,10.6529,9.0954,9.7210,90.0015,89.9996,...,2.125,3.375,3.525,2.125,-39.874295,0.335601,0.329533,0.372616,-0.235314,4.842888


### Categorize spacegroups

In [7]:
# transform encoding
def encode_spacegroup(X):
    # 1-2 triclinic
    # 3-15 monoclinic
    # 16-74 orthorhombic
    # 75-142 tetragonal
    # 143-167 trigonal
    # 168-194 hexagonal
    # 195-230 cubic
    # [ 33 194 227 167 206  12] are the possible spacegroup values
    # onehot encode each separately
    return pd.get_dummies(X, columns=["spacegroup"])

In [8]:
X_full = encode_spacegroup(X_full)

## Scale attributes to from 0 to 1

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_full)

X_full = scaler.transform(X_full)

In [10]:
X_full

array([[1.        , 0.625     , 0.375     , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.625     , 0.375     , ..., 1.        , 0.        ,
        0.        ],
       [0.42857143, 0.8125    , 0.1875    , ..., 0.        , 0.        ,
        1.        ],
       ...,
       [1.        , 0.4375    , 0.5625    , ..., 0.        , 1.        ,
        0.        ],
       [1.        , 0.3125    , 0.1875    , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.875     , 0.0938    , ..., 0.        , 1.        ,
        0.        ]])

### Separate into Training and validation

In [11]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

trainSize = 0.85

############## Separate Train into Validation and Train for Formation energy ####################
X_train_energ, X_val_energ, y_train_energ, y_val_energ = train_test_split(X_full, y_fe, train_size=trainSize, random_state=1)


############## Separate Train into Validation and Train for Band Gap ####################
X_train_gap, X_val_gap, y_train_gap, y_val_gap = train_test_split(X_full, y_be, train_size=trainSize, random_state=1)


# Model training

In [24]:
from sklearn.metrics import mean_squared_log_error
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

def rmsle_exp(y_true, y_pred, **kwargs):
    return np.sqrt(mean_squared_log_error(np.expm1(y_true), np.expm1(y_pred))) 
rmsle_scorer_exp = sklearn.metrics.make_scorer(rmsle_exp, greater_is_better=False)

## KNN

#### Formation energy

In [13]:
#################### KNN ##############################
from sklearn.neighbors import KNeighborsRegressor
results_df = pd.DataFrame(columns=['model',"Neigbors","Metric","weight",'trainError', 'valError'])

for metr in ["manhattan", "minkowski"]:
    for i in range(3, 71, 2):
        for weight in ['uniform', 'distance']:
            knn = KNeighborsRegressor(n_neighbors=i).fit(X_train_energ, y_train_energ)

#            print("Neighbours " + str(i))
#            print("Metric " + metr)

            trainError = rmsle(y_train_energ, knn.predict(X_train_energ))
            valError = rmsle(y_val_energ, knn.predict(X_val_energ))

            results_df = results_df.append({'model': 'KNN',"Neigbors":i,"Metric":metr,"weight":weight,"trainError":trainError, 'valError':valError, 'deltaErrors':abs(trainError-valError)}, ignore_index=True)



In [14]:
results_df.sort_values(by=["valError", "deltaErrors"])

,model,Neigbors,Metric,weight,min_samples_split,trainError,valError,deltaErrors
4,KNN,7,manhattan,uniform,NaN,0.029758,0.033206,0.003448
5,KNN,7,manhattan,distance,NaN,0.029758,0.033206,0.003448
72,KNN,7,minkowski,uniform,NaN,0.029758,0.033206,0.003448
73,KNN,7,minkowski,distance,NaN,0.029758,0.033206,0.003448
6,KNN,9,manhattan,uniform,NaN,0.030828,0.033243,0.002415
...,...,...,...,...,...,...,...,...
133,KNN,67,minkowski,distance,NaN,0.043612,0.043510,0.000101
66,KNN,69,manhattan,uniform,NaN,0.043943,0.044013,0.000070
67,KNN,69,manhattan,distance,NaN,0.043943,0.044013,0.000070
134,KNN,69,minkowski,uniform,NaN,0.043943,0.044013,0.000070


#### Band Gap

In [15]:
#################### KNN ##############################
from sklearn.neighbors import KNeighborsRegressor
results_df = pd.DataFrame(columns=['model',"Neigbors","Metric","weight",'trainError', 'valError'])

for metr in ["manhattan", "minkowski"]:
    for i in range(3, 71, 2):
        for weight in ['uniform', 'distance']:
            knn = KNeighborsRegressor(n_neighbors=i).fit(X_train_gap, y_train_gap)

#            print("Neighbours " + str(i))
#            print("Metric " + metr)

            trainError = rmsle(y_train_gap, knn.predict(X_train_gap))
            valError = rmsle(y_val_gap, knn.predict(X_val_gap))

            results_df = results_df.append({'model': 'KNN',"Neigbors":i,"Metric":metr,"weight":weight,"trainError":trainError, 'valError':valError, 'deltaErrors':abs(trainError-valError)}, ignore_index=True)



In [16]:
results_df.sort_values(by=["valError", "deltaErrors"])

,model,Neigbors,Metric,weight,min_samples_split,trainError,valError,deltaErrors
4,KNN,7,manhattan,uniform,NaN,0.087693,0.091661,0.003968
5,KNN,7,manhattan,distance,NaN,0.087693,0.091661,0.003968
72,KNN,7,minkowski,uniform,NaN,0.087693,0.091661,0.003968
73,KNN,7,minkowski,distance,NaN,0.087693,0.091661,0.003968
2,KNN,5,manhattan,uniform,NaN,0.083395,0.095135,0.011740
...,...,...,...,...,...,...,...,...
133,KNN,67,minkowski,distance,NaN,0.146628,0.151854,0.005225
66,KNN,69,manhattan,uniform,NaN,0.147615,0.153061,0.005446
67,KNN,69,manhattan,distance,NaN,0.147615,0.153061,0.005446
134,KNN,69,minkowski,uniform,NaN,0.147615,0.153061,0.005446


## SVM

#### Formation energy

In [35]:

################ SVC ##################################
from sklearn.svm import SVR

kernels = ['poly', 'rbf']
gammas=['scale', 'auto']
for ker in kernels:
    for gam in gammas:
        svm = SVR(kernel=ker, gamma=gam).fit(X_train_energ, y_train_energ)
        trainError = rmsle(y_train_energ, abs(svm.predict(X_train_energ)))
        valError = rmsle(y_val_energ, abs(svm.predict(X_val_energ)))
        print("Kernel: " + ker)
        print("Gamma: "+ gam)
        print("Training error: "+str(trainError))
        print("Validation error: "+ str(valError))
############### Estimate AUC #####################################

#probs = pd.DataFrame(svm.predict_proba(X_val2))[1]
#fpr, tpr, thresholds = metrics.roc_curve(y_val2, probs)
#print("Validation AUC: "+ str(metrics.auc(fpr, tpr)))

############## Evaluate Test #######################

#Create dataframe for output
#probs = pd.DataFrame(svm.predict_proba(test_df))[1]
#out = pd.DataFrame({'id':test_df.index,'y':probs})
#out.to_csv("SVM_rbf.csv", index=False)


Kernel: poly
Gamma: scale
Training error: 0.046069169727958076
Validation error: 0.046727083065543504
Kernel: poly
Gamma: auto
Training error: 0.05414831683786645
Validation error: 0.05364235979638728
Kernel: rbf
Gamma: scale
Training error: 0.04645671148891898
Validation error: 0.04642970080484417
Kernel: rbf
Gamma: auto
Training error: 0.04682473499777388
Validation error: 0.04619419409996547


#### Band gap

In [34]:

################ SVC ##################################
from sklearn.svm import SVR

kernels = ['poly', 'rbf']
gammas=['scale', 'auto']
for ker in kernels:
    for gam in gammas:
        svm = SVR(kernel=ker, gamma=gam).fit(X_train_gap, y_train_gap)
        trainError = rmsle(y_train_gap, abs(svm.predict(X_train_gap)))
        valError = rmsle(y_val_gap, abs(svm.predict(X_val_gap)))
        print("Kernel: " + ker)
        print("Gamma: "+ gam)
        print("Training error: "+str(trainError))
        print("Validation error: "+ str(valError))
############### Estimate AUC #####################################

#probs = pd.DataFrame(svm.predict_proba(X_val2))[1]
#fpr, tpr, thresholds = metrics.roc_curve(y_val2, probs)
#print("Validation AUC: "+ str(metrics.auc(fpr, tpr)))

############## Evaluate Test #######################

#Create dataframe for output
#probs = pd.DataFrame(svm.predict_proba(test_df))[1]
#out = pd.DataFrame({'id':test_df.index,'y':probs})
#out.to_csv("SVM_rbf.csv", index=False)


Kernel: poly
Gamma: scale
Training error: 0.07584564240632702
Validation error: 0.07156753875472792
Kernel: poly
Gamma: auto
Training error: 0.1627855310123912
Validation error: 0.16701195291483512
Kernel: rbf
Gamma: scale
Training error: 0.08132562215620576
Validation error: 0.07411458320991757
Kernel: rbf
Gamma: auto
Training error: 0.09657485671680917
Validation error: 0.08403921755449209
